# Animating NFL Plays in Plotly

I was inspired by [Nick Wan's Matplotlib GIF visualization](https://www.kaggle.com/code/nickwan/animated-gif-for-plays-python) to build an animated visualization with Plotly that can animate plays but also show details of the players on the field for a play and additional information like game time, down and distance. Hopefully this is a helpful tool for others during EDA steps of the process.

In [36]:
import pandas as pd
import numpy as np
import math
import plotly.graph_objects as go
import plotly.figure_factory as ff
import sympy as sp
from sympy import I, sqrt, symbols

# Read In csvss
games = pd.read_csv("nfl-big-data-bowl-2024/games.csv")
plays = pd.read_csv("nfl-big-data-bowl-2024/plays.csv")
players = pd.read_csv("nfl-big-data-bowl-2024/players.csv")
week1 = pd.read_csv("nfl-big-data-bowl-2024/tracking_week_1.csv")
tacklesData = pd.read_csv("nfl-big-data-bowl-2024/tackles.csv")
joined_all = pd.merge(games,plays,how="inner",on = "gameId")
joined_all = pd.merge(joined_all,week1,how="inner",on=["gameId","playId"])
# left join on players to keep football records
joined_all = pd.merge(joined_all,players,how="left",on = "nflId")
play_focus = 2184
focused_df = joined_all[(joined_all.playId==play_focus)]
focused_df

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore,playId,...,dis,o,dir,event,height,weight,birthDate,collegeName,position,displayName_y
0,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,0.17,286.97,34.95,pass_arrived,6-5,325.0,1988-06-06,Indiana,G,Rodger Saffold
1,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,0.14,296.88,23.83,NaN,6-5,325.0,1988-06-06,Indiana,G,Rodger Saffold
2,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,0.12,298.55,7.42,NaN,6-5,325.0,1988-06-06,Indiana,G,Rodger Saffold
3,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,0.10,298.55,350.77,NaN,6-5,325.0,1988-06-06,Indiana,G,Rodger Saffold
4,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,0.05,299.99,325.69,NaN,6-5,325.0,1988-06-06,Indiana,G,Rodger Saffold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,0.35,NaN,NaN,tackle,NaN,NaN,NaN,NaN,NaN,NaN
1008,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,0.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1009,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,0.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
focused_df.columns

Index(['gameId', 'season', 'week', 'gameDate', 'gameTimeEastern',
       'homeTeamAbbr', 'visitorTeamAbbr', 'homeFinalScore',
       'visitorFinalScore', 'playId', 'ballCarrierId',
       'ballCarrierDisplayName', 'playDescription', 'quarter', 'down',
       'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineSide',
       'yardlineNumber', 'gameClock', 'preSnapHomeScore',
       'preSnapVisitorScore', 'passResult', 'passLength', 'penaltyYards',
       'prePenaltyPlayResult', 'playResult', 'playNullifiedByPenalty',
       'absoluteYardlineNumber', 'offenseFormation', 'defendersInTheBox',
       'passProbability', 'preSnapHomeTeamWinProbability',
       'preSnapVisitorTeamWinProbability', 'homeTeamWinProbabilityAdded',
       'visitorTeamWinProbilityAdded', 'expectedPoints', 'expectedPointsAdded',
       'foulName1', 'foulName2', 'foulNFLId1', 'foulNFLId2', 'nflId',
       'displayName_x', 'frameId', 'time', 'jerseyNumber', 'club',
       'playDirection', 'x', 'y', 's', 'a', 'dis',

# Simple Plotly graph

If all we want to do is plot out a simple frame for a play in plotly it is very straightforward. We can see the 22 players and the football but dont really have good context as to where they are on the field or what the other situational variables are like down, distance or time in the game. We also aretn making good use of the mouse over features in plotly since right now all they are showing is the x,y position of the player.

In [39]:
fig = go.Figure( layout_yaxis_range=[0,53.3], layout_xaxis_range=[0,120])

colors = {
            "BUF":'#D50A0A',
            "LA":'#003594',
            "football":'#CBB67C'
}

for team in focused_df.club.unique():
    plot_df = focused_df[(focused_df.club==team)&(focused_df.frameId==1)]
    fig.add_trace(go.Scatter(x=plot_df["x"], y=plot_df["y"],mode = 'markers',marker_color=colors[team],name=team))
fig.show()

# Plot predict reachable Area (Ally only)

In [8]:
target_df = focused_df[(focused_df.team == "TB")&(focused_df.frameId==targetFrame)]

x = target_df["x"]
x_QB = target_df[target_df['officialPosition'] == 'QB']["x"]
x_not_QB = target_df[target_df['officialPosition'] != 'QB']["x"]
x0 = target_df["x0"]
x1 = target_df["x1"]

y = target_df["y"]
y_QB = target_df[target_df['officialPosition'] == 'QB']["y"]
y_not_QB = target_df[target_df['officialPosition'] != 'QB']["y"]
y0 = target_df["y0"]
y1 = target_df["y1"]

u = target_df["reachable_area_cx"] - target_df["x"]
v = target_df["reachable_area_cy"] - target_df["y"]

In [9]:
# あるチームのあるフレームを抽出
target_df = focused_df[(focused_df.team == "TB") & (focused_df.frameId == targetFrame)]

# レイアウトの設定
layout = go.Layout(
        autosize=False,
        width=1200,
        height=600,
        xaxis=dict(range=[0, 120], autorange=False, tickmode='array',tickvals=np.arange(10, 111, 5).tolist(),showticklabels=False),
        yaxis=dict(range=[0, 53.3], autorange=False,showgrid=False,showticklabels=False),

        plot_bgcolor='#00B140',
        # Create title and add play description at the bottom of the chart for better visual appeal
    )

# 矢印の描画
fig = ff.create_quiver(x, y, u, v,
                       scale=1,
                       arrow_scale=0.1,
                       name='reachable center vec',
                       line_width=1,
                       line_color = 'white'
                       )

# 点の描画
fig.add_trace(go.Scatter(x=x_not_QB, y=y_not_QB, mode='markers', marker_color=colors["TB"], name="TB"))
fig.add_trace(go.Scatter(x=x_QB, y=y_QB, mode='markers',marker_symbol='star', marker_color=colors["TB"], name="QB"))


for xi, yi, xi0, yi0, xi1, yi1 in zip(x, y, x0, y0, x1, y1):
    if not np.isnan(xi0):
        fig.add_shape(type="circle",
                      xref="x", yref="y",
                      x0=xi0, y0=yi0,
                      x1=xi1, y1=yi1,
                      line_color="white",
                      )

# layoutを再度設定
fig.update_layout(layout) 

fig.show()

fig_ally = fig


# Plot predict reachable Area (Enemy only)

In [10]:
target_team = "DAL"

target_df = focused_df[(focused_df.team == target_team)&(focused_df.frameId==targetFrame)]

x = target_df["x"]
x_QB = target_df[target_df['officialPosition'] == 'QB']["x"]
x_not_QB = target_df[target_df['officialPosition'] != 'QB']["x"]
x0 = target_df["x0"]
x1 = target_df["x1"]

y = target_df["y"]
y_QB = target_df[target_df['officialPosition'] == 'QB']["y"]
y_not_QB = target_df[target_df['officialPosition'] != 'QB']["y"]
y0 = target_df["y0"]
y1 = target_df["y1"]

u = target_df["reachable_area_cx"] - target_df["x"]
v = target_df["reachable_area_cy"] - target_df["y"]

fig = go.Figure()

# レイアウトの設定
layout = go.Layout(
        autosize=False,
        width=1200,
        height=600,
        xaxis=dict(range=[0, 120], autorange=False, tickmode='array',tickvals=np.arange(10, 111, 5).tolist(),showticklabels=False),
        yaxis=dict(range=[0, 53.3], autorange=False,showgrid=False,showticklabels=False),

        plot_bgcolor='#00B140',
        # Create title and add play description at the bottom of the chart for better visual appeal
    )

# 矢印の描画
fig_quiver = ff.create_quiver(x, y, u, v,
                       scale=1,
                       arrow_scale=0.1,
                       name='reachable center vec',
                       line_width=1,
                       line_color = 'white'
                       )

fig.add_traces(data = fig_quiver.data)

# 点の描画
fig.add_trace(go.Scatter(x=x_not_QB, y=y_not_QB, mode='markers', marker_color=colors[target_team], name=target_team))
fig.add_trace(go.Scatter(x=x_QB, y=y_QB, mode='markers',marker_symbol='star', marker_color=colors[target_team], name=target_team))


for xi, yi, xi0, yi0, xi1, yi1 in zip(x, y, x0, y0, x1, y1):
    if not np.isnan(xi0):
        fig.add_shape(type="circle",
                      xref="x", yref="y",
                      x0=xi0, y0=yi0,
                      x1=xi1, y1=yi1,
                      line_color="white",
                      )

# layoutを再度設定
fig.update_layout(layout) 

fig.show()

fig_enemy = fig

# Plot reachable Area (Ally, Enemy, and football)

In [11]:
team_list = ["TB", "DAL", "football"]
fig = go.Figure()

for target_team in team_list:
    target_df = focused_df[(focused_df.team == target_team)&(focused_df.frameId==targetFrame)]

    x = target_df["x"]
    x_QB = target_df[target_df['officialPosition'] == 'QB']["x"]
    x_not_QB = target_df[target_df['officialPosition'] != 'QB']["x"]
    x0 = target_df["x0"]
    x1 = target_df["x1"]

    y = target_df["y"]
    y_QB = target_df[target_df['officialPosition'] == 'QB']["y"]
    y_not_QB = target_df[target_df['officialPosition'] != 'QB']["y"]
    y0 = target_df["y0"]
    y1 = target_df["y1"]

    u = target_df["reachable_area_cx"] - target_df["x"]
    v = target_df["reachable_area_cy"] - target_df["y"]


    # 予測到達点矢印の描画
    fig_quiver = ff.create_quiver(x, y, u, v,
                           scale=1,
                           arrow_scale=0.1,
                           name='reachable center vec',
                           line_width=1,
                           line_color = colors[target_team]
                           )
    fig.add_traces(data = fig_quiver.data)

    # 点の描画
    fig.add_trace(go.Scatter(x=x_not_QB, y=y_not_QB, mode='markers', marker_color=colors[target_team], name=target_team))
    fig.add_trace(go.Scatter(x=x_QB, y=y_QB, mode='markers',marker_symbol='star', marker_color=colors[target_team], name=target_team))

    # 領域の描画     
    for xi, yi, xi0, yi0, xi1, yi1 in zip(x, y, x0, y0, x1, y1):
        if not np.isnan(xi0):
            fig.add_shape(type="circle",
                          xref="x", yref="y",
                          x0=xi0, y0=yi0,
                          x1=xi1, y1=yi1,
                          line_color=colors[target_team],
                          )

# レイアウトの設定
layout = go.Layout(
        autosize=False,
        width=1200,
        height=600,
        xaxis=dict(range=[0, 120], autorange=False, tickmode='array',tickvals=np.arange(10, 111, 5).tolist(),showticklabels=False),
        yaxis=dict(range=[0, 53.3], autorange=False,showgrid=False,showticklabels=False),

        plot_bgcolor='#00B140',
        # Create title and add play description at the bottom of the chart for better visual appeal
    )

# layoutを設定
fig.update_layout(layout) 

# figの表示
fig.show()

# Complex Plotly Graph

For my more complex graph I set a few goals. 

**Goals:**
- Animate all frames in a play
- Color player dots with the actual team colors instead of generic home/away colors or having to set different colors each time I run the code
- Add Down, Distance and Situational information to the Animation
- Add Player information in hover/tooltip
- Add Play Description

### Get Colors for every team

I found a website online with color codes for each NFL team to help personalize the Animations a bit. 

Color Codes Found on the following sites: 

- https://teamcolorcodes.com/nfl-team-color-codes/

- https://www.schemecolor.com/american-football.php

Note: Not all color combo's are tested so you may get a weird combination. I could probably work out some color difference and switch to an alternate if they are too close but this is a first pass at it

In [61]:
colors = {
    'ARI':"#97233F", 
    'ATL':"#A71930", 
    'BAL':'#241773', 
    'BUF':"#00338D", 
    'CAR':"#0085CA", 
    'CHI':"#C83803", 
    'CIN':"#FB4F14", 
    'CLE':"#311D00", 
    'DAL':'#003594',
    'DEN':"#FB4F14", 
    'DET':"#0076B6", 
    'GB':"#203731", 
    'HOU':"#03202F", 
    'IND':"#002C5F", 
    'JAX':"#9F792C", 
    'KC':"#E31837", 
    'LA':"#003594", 
    'LAC':"#0080C6", 
    'LV':"#000000",
    'MIA':"#008E97", 
    'MIN':"#4F2683", 
    'NE':"#002244", 
    'NO':"#D3BC8D", 
    'NYG':"#0B2265", 
    'NYJ':"#125740", 
    'PHI':"#004C54", 
    'PIT':"#FFB612", 
    'SEA':"#69BE28", 
    'SF':"#AA0000",
    'TB':'#D50A0A', 
    'TEN':"#4B92DB", 
    'WAS':"#5A1414", 
    'football':'#CBB67C'
}

### Play Animation Function

In [57]:
#TODO:
# Add Useful Player Labels
# Add ability to keep or remove the extra stuff (line markers etc)
# Add Team Colors to dict
# Turn into Function 

def animate_play(tracking_df, play_df,players,tacklesData, gameId,playId):
    selected_play_df = play_df[(play_df.playId==playId)&(play_df.gameId==gameId)].copy()
    
    tracking_players_df = pd.merge(tracking_df,players,how="left",on = "nflId")
    tracking_players_df = pd.merge(tracking_players_df,tacklesData,how="left",on = ["nflId","playId","gameId"])
    selected_tracking_df = tracking_players_df[(tracking_players_df.playId==playId)&(tracking_players_df.gameId==gameId)].copy()

    sorted_frame_list = selected_tracking_df.frameId.unique()
    sorted_frame_list.sort()

    # get play General information 
    line_of_scrimmage = selected_play_df.absoluteYardlineNumber.values[0]
    first_down_marker = line_of_scrimmage + selected_play_df.yardsToGo.values[0]
    down = selected_play_df.down.values[0]
    quarter = selected_play_df.quarter.values[0]
    gameClock = selected_play_df.gameClock.values[0]
    playDescription = selected_play_df.playDescription.values[0]
    # Handle case where we have a really long Play Description and want to split it into two lines
    if len(playDescription.split(" "))>15 and len(playDescription)>115:
        playDescription = " ".join(playDescription.split(" ")[0:16]) + "<br>" + " ".join(playDescription.split(" ")[16:])

    # initialize plotly start and stop buttons for animation
    updatemenus_dict = [
        {
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 100, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 0}}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                      "mode": "immediate",
                                      "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }
    ]
    # initialize plotly slider to show frame position in animation
    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": []
    }


    frames = []
    for frameId in sorted_frame_list:
        data = []
        # Add Numbers to Field 
        data.append(
            go.Scatter(
                x=np.arange(20,110,10), 
                y=[5]*len(np.arange(20,110,10)),
                mode='text',
                text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
                textfont_size = 30,
                textfont_family = "Courier New, monospace",
                textfont_color = "#ffffff",
                showlegend=False,
                hoverinfo='none'
            )
        )
        data.append(
            go.Scatter(
                x=np.arange(20,110,10), 
                y=[53.5-5]*len(np.arange(20,110,10)),
                mode='text',
                text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
                textfont_size = 30,
                textfont_family = "Courier New, monospace",
                textfont_color = "#ffffff",
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Add line of scrimage 
        data.append(
            go.Scatter(
                x=[line_of_scrimmage,line_of_scrimmage], 
                y=[0,53.5],
                line_dash='dash',
                line_color='blue',
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Add First down line 
        data.append(
            go.Scatter(
                x=[first_down_marker,first_down_marker], 
                y=[0,53.5],
                line_dash='dash',
                line_color='yellow',
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Plot Players
        for team in selected_tracking_df.club.unique():
            plot_df = selected_tracking_df[(selected_tracking_df.club==team)&(selected_tracking_df.frameId==frameId)].copy()
            if team != "football":
                hover_text_array=[]
                for nflId in plot_df.nflId:
                    selected_player_df = plot_df[plot_df.nflId==nflId]
                    hover_text_array.append("nflId:{}".format(selected_player_df["nflId"].values[0]))
                data.append(go.Scatter(x=plot_df["x"], y=plot_df["y"],mode = 'markers',marker_color=colors[team],name=team,hovertext=hover_text_array,hoverinfo="text"))
            else:
                data.append(go.Scatter(x=plot_df["x"], y=plot_df["y"],mode = 'markers',marker_color=colors[team],name=team,hoverinfo='none'))
        # add frame to slider
        slider_step = {"args": [
            [frameId],
            {"frame": {"duration": 100, "redraw": False},
             "mode": "immediate",
             "transition": {"duration": 0}}
        ],
            "label": str(frameId),
            "method": "animate"}
        sliders_dict["steps"].append(slider_step)
        frames.append(go.Frame(data=data, name=str(frameId)))

    scale=10
    layout = go.Layout(
        autosize=False,
        width=120*scale,
        height=60*scale,
        xaxis=dict(range=[0, 120], autorange=False, tickmode='array',tickvals=np.arange(10, 111, 5).tolist(),showticklabels=False),
        yaxis=dict(range=[0, 53.3], autorange=False,showgrid=False,showticklabels=False),

        plot_bgcolor='#00B140',
        # Create title and add play description at the bottom of the chart for better visual appeal
        title=f"GameId: {gameId}, PlayId: {playId}<br>{gameClock} {quarter}Q"+"<br>"*19+f"{playDescription}",
        updatemenus=updatemenus_dict,
        sliders = [sliders_dict]
    )

    fig = go.Figure(
        data=frames[0]["data"],
        layout= layout,
        frames=frames[1:]
    )
    # Create First Down Markers 
    for y_val in [0,53]:
        fig.add_annotation(
                x=first_down_marker,
                y=y_val,
                text=str(down),
                showarrow=False,
                font=dict(
                    family="Courier New, monospace",
                    size=16,
                    color="black"
                    ),
                align="center",
                bordercolor="black",
                borderwidth=2,
                borderpad=4,
                bgcolor="#ff7f0e",
                opacity=1
                )

    return fig


# Examples 

In [68]:
animate_play(week1,plays,players,tacklesData,2022090800,2307).show()

In [65]:
joined_all[joined_all["passResult"] == "C"]["playId"].unique()

array([2184, 2307,  236, 2336, 2860, 3166,  867,  617, 2232,  980, 1712,
       2208, 3383,  364, 3092,  692, 2572, 3190,   56,  212, 2527, 2688,
       2599, 3513,  122,  593,  818, 1030, 1967, 3489,  438, 2815,  414,
       3407, 3576,  167, 3121, 2979, 1102,  486, 2934,  775, 1230, 1358,
       1836, 1563, 2884, 1334,  569, 3431,  646, 1286, 1887, 1359,  121,
       1769, 3554, 1552, 3177, 3310, 1672, 2159,  522, 1485, 3119,  679,
       4307, 4266, 1017, 2245, 2090, 4032, 3446, 1231, 3526,  783, 1928,
       3578, 2135,  166, 2555,  458, 1335, 2616, 4102, 2303, 3475, 2374,
       3355, 3148, 3647, 2960, 3591, 1945, 1785, 1744,  109, 3101, 4068,
       3263, 3545, 1691,  542, 2407, 3862, 1384, 4104, 2951, 4150, 2341,
        748, 3789,  662, 1077, 3454, 3510, 2832, 1720,   85,  184, 3707,
        850,  382,  272, 3336, 1517, 2302, 2103, 3783, 2894, 2639, 3628,
       2532, 3695, 1133, 3859, 1406,  756, 1631, 2132, 2759,  189,  239,
       1171, 4686, 2058, 4872, 2740, 2483,  111, 50

In [67]:
#Select random plays from 5 games
import random
for gameId in week1.gameId.unique()[0:5].tolist():
    plays_list = week1[week1.gameId==gameId].playId.unique().tolist()
    playId = random.choice(plays_list)
    animate_play(week1,plays,players,tacklesData,gameId,playId).show()

In [27]:
team_list = ["TB", "DAL"]
fig = go.Figure()

for target_team in team_list:
    target_df = focused_df[(focused_df.team == target_team)&(focused_df.frameId==targetFrame)]

    x = target_df["x"]
    x_QB = target_df[target_df['officialPosition'] == 'QB']["x"]
    x_not_QB = target_df[target_df['officialPosition'] != 'QB']["x"]
    x0 = target_df["x0"]
    x1 = target_df["x1"]

    y = target_df["y"]
    y_QB = target_df[target_df['officialPosition'] == 'QB']["y"]
    y_not_QB = target_df[target_df['officialPosition'] != 'QB']["y"]
    y0 = target_df["y0"]
    y1 = target_df["y1"]

    u = target_df["reachable_area_cx"] - target_df["x"]
    v = target_df["reachable_area_cy"] - target_df["y"]


    # 予測到達点矢印の描画
    fig_quiver = ff.create_quiver(x, y, u, v,
                           scale=1,
                           arrow_scale=0.1,
                           name='reachable center vec',
                           line_width=1,
                           line_color = colors[target_team]
                           )
    fig.add_traces(data = fig_quiver.data)

    # 選手点の描画
    fig.add_trace(go.Scatter(x=x_not_QB, y=y_not_QB, mode='markers', marker_color=colors[target_team], name=target_team))
    fig.add_trace(go.Scatter(x=x_QB, y=y_QB, mode='markers',marker_symbol='star', marker_color=colors[target_team], name=target_team))

    # 領域の描画     
    for xi, yi, xi0, yi0, xi1, yi1 in zip(x, y, x0, y0, x1, y1):
        if not np.isnan(xi0):
            fig.add_shape(type="circle",
                          xref="x", yref="y",
                          x0=xi0, y0=yi0,
                          x1=xi1, y1=yi1,
                          line_color=colors[target_team],
                          )

target_team = "football"
# footballの軌道描画
x_fb_0 = focused_df[(focused_df.team == target_team)&(focused_df.frameId==targetFrame)]["x"].values[0]
y_fb_0 = focused_df[(focused_df.team == target_team)&(focused_df.frameId==targetFrame)]["y"].values[0]
x_fb_1 = focused_df[(focused_df.team == target_team)&(focused_df.frameId==targetFrame + 1)]["x"].values[0]
y_fb_1 = focused_df[(focused_df.team == target_team)&(focused_df.frameId==targetFrame + 1)]["y"].values[0]

y_orb_0 = 0
y_orb_1 = 53.3
x_orb_0 = x_fb_0 + (x_fb_0 - x_fb_1) * (y_orb_0 - y_fb_0) / (y_fb_0 - y_fb_1)
x_orb_1 = x_fb_0 + (x_fb_0 - x_fb_1) * (y_orb_1 - y_fb_0) / (y_fb_0 - y_fb_1)

fig.add_trace(go.Scatter(x = [x_orb_0, x_orb_1], y = [y_orb_0, y_orb_1], line = dict(color = 'yellow',  dash = "dot"), name = "ball orbit"))


 #　円と直線の交点の描画
cross_x0 = focused_df[focused_df["isCrossed"] == 1]["cross_x0"].tolist()
cross_x1 = focused_df[focused_df["isCrossed"] == 1]["cross_x1"].tolist()
cross_y0 = focused_df[focused_df["isCrossed"] == 1]["cross_y0"].tolist()
cross_y1 = focused_df[focused_df["isCrossed"] == 1]["cross_y1"].tolist()

cross_x = cross_x0
cross_x.extend(cross_x1)

cross_y = cross_y0
cross_y.extend(cross_y1)

fig.add_trace(go.Scatter(x=cross_x, y=cross_y, mode='markers', marker_color="black", name="cross_point"))


# レイアウトの設定
layout = go.Layout(
        autosize=False,
        width=1200,
        height=600,
        xaxis=dict(range=[0, 120], autorange=False, tickmode='array',tickvals=np.arange(10, 111, 5).tolist(),showticklabels=False),
        yaxis=dict(range=[0, 53.3], autorange=False,showgrid=False,showticklabels=False),

        plot_bgcolor='#00B140',
        # Create title and add play description at the bottom of the chart for better visual appeal
    )

# layoutを設定
fig.update_layout(layout) 

# figの表示
fig.show()

# make cross point list

In [17]:
focused_df[focused_df["isCrossed"]==1][["x", "y", "team", "cross_x0", "cross_x1"]]
player_cross_df = focused_df[focused_df["isCrossed"]==1][["team", "cross_x0", "cross_x1"]]

player_cross_df

,team,cross_x0,cross_x1
555,TB,49.896318,67.541411
598,DAL,52.190460,69.172424
770,DAL,43.185407,52.684624
899,DAL,31.752871,33.622994


In [18]:
cross_point_list = focused_df[focused_df["isCrossed"]==1]["cross_x0"].unique().tolist()
cross_point_list.extend(focused_df[focused_df["isCrossed"]==1]["cross_x1"].unique().tolist())
cross_point_list.sort()

# area categorize

In [19]:
area_df = pd.DataFrame(
    {
        "start_point" : cross_point_list[:-1],
        "end_point" : cross_point_list[1:],
        "check_point" : [(x + y) / 2 for x, y in zip(cross_point_list[:-1], cross_point_list[1:])],
        "enemy" : 0,
        "ally" : 0,
    }
)
area_df["length"] = abs(area_df["end_point"] - area_df["start_point"])

area_df

,start_point,end_point,check_point,enemy,ally,length
0,31.752871,33.622994,32.687932,0,0,1.870123
1,33.622994,43.185407,38.404200,0,0,9.562412
2,43.185407,49.896318,46.540863,0,0,6.710912
3,49.896318,52.190460,51.043389,0,0,2.294142
4,52.190460,52.684624,52.437542,0,0,0.494164
5,52.684624,67.541411,60.113017,0,0,14.856787
6,67.541411,69.172424,68.356917,0,0,1.631013


In [20]:
#  'possessionTeam', 'defensiveTeam'
possessionTeam = focused_df.possessionTeam[0]
defensiveTeam = focused_df.defensiveTeam[0]

def isInRange(cp, start, end):
    return start <= cp < end

def catArea(row):
    if (row["enemy"] > 0) & (row["ally"] > 0):
        return "E&A"
    elif row["enemy"] > 0:
        return "E"
    elif row["ally"] > 0:
        return "A"
    else:
        return "F"

for index, data in area_df.iterrows():
    cp = data["check_point"]
    for index_player, data_player in player_cross_df.iterrows():
        cross_range = sorted([data_player["cross_x0"], data_player["cross_x1"]])
        isInRange = cross_range[0] <= cp < cross_range[1]
        isEnemy = data_player["team"] == defensiveTeam
        isAlly = data_player["team"] == possessionTeam
        if not isInRange:
            continue
        if isEnemy:
            area_df.at[index, "enemy"] += 1
        else:
            area_df.at[index, "ally"] += 1

area_df["category"] = area_df.apply(catArea, axis=1)

area_df

,start_point,end_point,check_point,enemy,ally,length,category
0,31.752871,33.622994,32.687932,1,0,1.870123,E
1,33.622994,43.185407,38.404200,0,0,9.562412,F
2,43.185407,49.896318,46.540863,1,0,6.710912,E
3,49.896318,52.190460,51.043389,1,1,2.294142,E&A
4,52.190460,52.684624,52.437542,2,1,0.494164,E&A
5,52.684624,67.541411,60.113017,1,1,14.856787,E&A
6,67.541411,69.172424,68.356917,1,0,1.631013,E


In [21]:
area_dic = {
    "E" : 0,
    "A": 0,
    "E&A" : 0,
    "F" : 0
}

for idx, data in area_df.iterrows():
    area_dic[data["category"]] += data["length"]

In [22]:
pred = area_dic["A"] / (area_dic["E&A"] + area_dic["E"])

In [23]:
pred

0.0

In [24]:
area_dic

{'E': 10.212047989121498,
 'A': 0,
 'E&A': 17.645092433600745,
 'F': 9.562412387457286}